Author: Deepu Variyangattil


In [37]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import queue
import re
import csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
#driver = webdriver.Chrome(chrome_options=options)
driver = webdriver.Chrome(executable_path='F:\Stevens_Masters\Spring 19\Web Mining\Week 3\chromedriver_win32\chromedriver.exe')



In [97]:
driver.get("https://www.epicurious.com/search/")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [99]:

menu_object1=(driver.find_element_by_xpath("//h3[contains(text(),'Popular')]"))
menu_object2=(driver.find_element_by_xpath("//h3[contains(text(),'Meal & Course')]"))
menu_object3=(driver.find_element_by_xpath("//h3[contains(text(),'Dish Type')]"))
menu_object4=(driver.find_element_by_xpath("//h3[contains(text(),'Dietary Concerns')]"))
menu_object5=(driver.find_element_by_xpath("//div[@class='filter-group']//h3[contains(text(),'Ingredient')]"))
menu_object6=(driver.find_element_by_xpath("//h3[contains(text(),'Cuisine')]"))
menu_object7=(driver.find_element_by_xpath("//h3[contains(text(),'Holiday')]"))
menu_object8=(driver.find_element_by_xpath("//h3[contains(text(),'Technique')]"))

In [100]:
menu_object1

<selenium.webdriver.remote.webelement.WebElement (session="8d29a92721860900adcde6cd086111a9", element="0.8108463319414423-1")>

In [101]:
menu_object1.click()
time.sleep(5)

catgories=driver.find_elements_by_xpath("//div[@class='filter-item']/input[@type='checkbox']")

ids=[]
q=queue.Queue()
for cat in catgories:
    ids.append(cat.get_attribute('id'))

for i in ids: 
    a=driver.find_element_by_id(i)
    
    driver.execute_script("arguments[0].click();", a)
    time.sleep(5)
    receipe_Links=(driver.find_elements_by_xpath("//*[contains(@href, 'recipe')]"))
    for recipe_link in receipe_Links:
        
        q.put(recipe_link.get_attribute('href'))
    

    

In [102]:
q.qsize()

1496

In [103]:
mydata=[["RECIPE","INGREDIENTS","URL"]]
myFile = open("recipelist2.csv", 'w',newline='')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(mydata)

In [104]:
url_List=[]
while q.qsize()>0:
    url=q.get()
    if re.match('(.)+recipes\/food(.)+',url) is not None:
        if url not in url_List:
            url_List.append(url)
            try:
                
                driver.get(url)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                recipe_name=driver.find_element_by_xpath("//h1[@itemprop='name']").text
                ing_List=driver.find_elements_by_xpath("//li[@class='ingredient']")
                s=""
                for ingre in ing_List:
        
        
                    p=ingre.text
                    p=p.replace(',',"")
                    if s=="":
                        s=p
                    else:
                        s=s+","+p
            
                mydata=[[recipe_name,s,url]]
                myFile = open("recipelist2.csv", 'a',newline='',encoding="utf-8")
                with myFile:
                    writer = csv.writer(myFile)
                    writer.writerows(mydata)
            except Exception:
                driver.find_element_by_xpath("//*[@class='bx-close-x-adaptive-1']").click()
print("Queue finished")
myFile.close()    

Queue finished


In [105]:
q.qsize()

0

In [106]:
driver.close()

In [107]:
len(url_List)

59

In [108]:
url_List

['https://www.epicurious.com/recipes/food/views/big-batch-pancake-and-waffle-mix',
 'https://www.epicurious.com/recipes/food/views/crispy-fried-shallots',
 'https://www.epicurious.com/recipes/food/views/spinach-and-artichoke-melts',
 'https://www.epicurious.com/recipes/food/views/za-atar-chicken-with-garlicky-yogurt',
 'https://www.epicurious.com/recipes/food/views/roast-chicken-thighs-with-peas-and-mint',
 'https://www.epicurious.com/recipes/food/views/chile-and-citrus-rubbed-chicken-with-potatoes',
 'https://www.epicurious.com/recipes/food/views/chicken-saltimbocca-with-crunchy-pea-salad',
 'https://www.epicurious.com/recipes/food/views/ginger-scallion-ramen-noodles',
 'https://www.epicurious.com/recipes/food/views/cold-soba-noodles-with-jammy-eggs-and-peas',
 'https://www.epicurious.com/recipes/food/views/weeknight-steak-and-rice-noodle-salad',
 'https://www.epicurious.com/recipes/food/views/better-than-takeout-stir-fried-udon',
 'https://www.epicurious.com/recipes/food/views/chicke